In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pip install dmba

In [ ]:

import math
from pathlib import Path
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_curve, auc
import matplotlib.pylab as plt
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LinearRegression, Lasso, Ridge, LassoCV, BayesianRidge
from dmba import regressionSummary, classificationSummary, liftChart, gainsChart,exhaustive_search

In [ ]:
data=pd.read_csv("../input/boston-housing-dataset/HousingData.csv")
data.head()

In [ ]:
data.info()

In [ ]:
data.isnull().sum()

In [ ]:
data['CRIM'].fillna(data['CRIM'].mean(),inplace=True)
data['ZN'].fillna(data['ZN'].mean(),inplace=True)
data['CHAS'].fillna(data['CHAS'].mean(),inplace=True)
data['INDUS'].fillna(data['INDUS'].mean(),inplace=True)
data['AGE'].fillna(data['AGE'].mean(),inplace=True)
data['LSTAT'].fillna(data['LSTAT'].mean(),inplace=True)
data.isnull().sum()

In [ ]:
X=['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']
data.boxplot(column=X,figsize=(13,13))

In [ ]:
data.describe()

In [ ]:
data["B"] = np.where(data["B"] <344.10624999999993, 344.10624999999993,data['B'])
data['LSTAT']=np.where(data["LSTAT"] >30.58, 30.58,data['LSTAT'])
data['CRIM']=np.where(data["CRIM"] >8.9048325, 8.9048325,data['CRIM'])
data['ZN']=np.where(data["ZN"] >28.029835, 28.029835,data['ZN'])

In [ ]:
X=['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']
data.boxplot(column=X,figsize=(13,13))

In [ ]:
X=['CRIM','ZN','INDUS','CHAS','NOX','RM','AGE','DIS','RAD','TAX','PTRATIO','B','LSTAT']
corr = data[X].corr()
import seaborn as sns
#sns.heatmap(corr, xticklabels=corr.columns, yticklabels=corr.columns, vmin=-1, vmax=1, cmap='RdBu')    # Include information about values
fig, ax = plt.subplots()
fig.set_size_inches(11, 7)
sns.heatmap(corr, annot=True, fmt=".1f", cmap="RdBu", center=0, ax=ax)
plt.show()

In [ ]:
from dmba import regressionSummary, exhaustive_search
from dmba import backward_elimination, forward_selection, stepwise_selection
from dmba import adjusted_r2_score, AIC_score, BIC_score

In [ ]:
outcome='MEDV'
x = data[X]
y = data[outcome]
train_X, valid_X, train_y, valid_y = train_test_split(x, y, test_size=0.4,random_state=1)


def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    pred_y = model.predict(train_X[variables])
    # we negate as score is optimized to be as low as possible
    return -adjusted_r2_score(train_y, pred_y, model)

allVariables = train_X.columns
results = exhaustive_search(allVariables, train_model, score_model)

data = []
for result in results:
    model = result['model']
    variables = result['variables']
    AIC = AIC_score(train_y, model.predict(train_X[variables]), model)
    
    d = {'n': result['n'], 'r2adj': -result['score'], 'AIC': AIC}
    d.update({var: var in result['variables'] for var in allVariables})
    data.append(d)
pd.set_option('display.width', 100)
print(pd.DataFrame(data, columns=('n', 'r2adj', 'AIC') + tuple(sorted(allVariables))))
pd.reset_option('display.width')

In [ ]:
def train_model(variables):
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = backward_elimination(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

In [ ]:
def train_model(variables):
    if len(variables) == 0:
        return None
    model = LinearRegression()
    model.fit(train_X[variables], train_y)
    return model

def score_model(model, variables):
    if len(variables) == 0:
        return AIC_score(train_y, [train_y.mean()] * len(train_y), model, df=1)
    return AIC_score(train_y, model.predict(train_X[variables]), model)

best_model, best_variables = forward_selection(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

In [ ]:
best_model, best_variables = stepwise_selection(train_X.columns, train_model, score_model, verbose=True)

print(best_variables)

In [ ]:
m = LinearRegression() 
m.fit(x, y)
regressionSummary(valid_y, m.predict(valid_X))

In [ ]:
ridge = Ridge(normalize=True, alpha=1) 
ridge.fit(train_X, train_y) 
regressionSummary(valid_y, ridge.predict(valid_X))

In [ ]:
lasso = Lasso(normalize=True, alpha=1)
lasso.fit(train_X, train_y) 
regressionSummary(valid_y, lasso.predict(valid_X))

In [ ]:
lasso_cv = LassoCV(normalize=True, cv=5) 
lasso_cv.fit(train_X, train_y) 
regressionSummary(valid_y, lasso_cv.predict(valid_X))

In [ ]:
print('Multiple Regression Summary')
print(regressionSummary(valid_y, m.predict(valid_X)))
print("------------------------------------------------------")
print('Ridge Regression')
print(regressionSummary(valid_y, ridge.predict(valid_X)))
print("------------------------------------------------------")
print('Lasso Regression')
print(regressionSummary(valid_y, lasso.predict(valid_X)))
print("------------------------------------------------------")
print('Lasso CV Regression')
print(regressionSummary(valid_y, lasso_cv.predict(valid_X)))